# 检测和分析人脸

计算机视觉解决方案通常需要人工智能 (AI) 解决方案才能检测、分析或识别人脸。例如，假设零售公司 Northwind Traders 决定开设一家“智能商店”，通过 AI 服务监视商店以识别需要帮助的顾客，然后引导店员去帮助这些顾客。达成该目标的一种方法是执行面部检测和分析，也就是确定图像中是否有人脸，并在存在人脸时分析其特征。

![正在分析人脸的机器人](./images/face_analysis.jpg)

## 使用人脸认知服务来检测人脸

假设 Northwind Traders 想要创建的智能商店系统需要能够检测到顾客并分析其面部特征。在 Microsoft Azure 中，可以使用 Azure 认知服务中包含的“**人脸**”功能来做到这一点。

### 创建认知服务资源

首先，在 Azure 订阅中创建**认知服务**资源。

> **备注**：如果已有认知服务资源，则只需在 Azure 门户中打开其“**快速入门**”页面，然后将其密钥和终结点复制到下面的单元格中即可。否则，请按照以下步骤创建认知服务资源。

1. 在另一个浏览器标签页中，打开 Azure 门户 (https://portal.azure.com) 并使用 Microsoft 帐户登录。
2. 单击“**&#65291;创建资源**”按钮，搜索“*认知服务*”并以如下设置创建**认知服务**资源：
    - **订阅**：*你的 Azure 订阅*。
    - **资源组**：*选择或创建具有唯一名称的资源组*。
    - **区域**：*选择任何可用区域*：
    - **名称**：*输入一个唯一名称*。
    - **定价层**：中的机器人 S0
    - **我确认我已阅读并理解上述通知**：已选中。
3. 等待部署完成。然后转到认知服务资源，并单击“**概述**”页面上的链接以管理该服务的密钥。你将需要使用终结点和密钥从客户端应用程序连接到认知服务资源。

### 获取认知服务资源的密钥和终结点

要使用认知服务资源，客户端应用程序需要其终结点和身份验证密钥：

1. 进入 Azure 门户，在认知服务资源的“**密钥和终结点**”页面上复制资源的“**Key1**”，并将其粘贴到以下代码中，替换“**YOUR_COG_KEY**”。

2. 复制资源的**终结点**，并将其粘贴到以下代码中，替换“**YOUR_COG_ENDPOINT**”。

3. 通过单击位于单元格左上角的“运行单元格”<span>&#9655;</span> 按钮运行下方单元格中的代码。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

现在你已准备好认知服务资源，可以使用人脸服务来检测商店中的人脸。

运行下面的代码单元格以查看示例。

In [ ]:
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from python_code import faces
import os
%matplotlib inline

# Create a face detection client.
face_client = FaceClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

# Open an image
image_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces)

系统会为检测到的每张人脸分配一个唯一的 ID，以便应用程序识别检测到的每张人脸。

运行下面的单元格以查看更多购物者人脸对应的 ID。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_stream = open(image_path, "rb")

# Detect faces
detected_faces = face_client.face.detect_with_stream(image=image_stream)

# Display the faces (code in python_code/faces.py)
faces.show_faces(image_path, detected_faces, show_id=True)

## 分析面部特征

人脸服务提供的功能远不止检测人脸。它还可通过分析面部特征和表情来判断年龄和情绪状态。例如，请运行下面的代码来分析购物者的面部特征。

In [ ]:
# Open an image
image_path = os.path.join('data', 'face', 'store_cam1.jpg')
image_stream = open(image_path, "rb")

# Detect faces and specified facial attributes
attributes = ['age', 'emotion']
detected_faces = face_client.face.detect_with_stream(image=image_stream, return_face_attributes=attributes)

# Display the faces and attributes (code in python_code/faces.py)
faces.show_face_attributes(image_path, detected_faces)

根据图像中检测到的顾客的情绪评分，这位顾客似乎对购物体验非常满意。

## 查找相似人脸 

为检测到的每张人脸创建的人脸 ID 用于单独识别其人脸检测信息。通过使用这些 ID，可以将检测到的人脸与之前检测到的人脸进行比较，并找到具有相似特征的人脸。

例如，请运行下面的单元格，将一张图像中的某位购物者与另一张图像中的购物者们进行比较，并找到与之匹配的人脸。

In [ ]:
# Get the ID of the first face in image 1
image_1_path = os.path.join('data', 'face', 'store_cam3.jpg')
image_1_stream = open(image_1_path, "rb")
image_1_faces = face_client.face.detect_with_stream(image=image_1_stream)
face_1 = image_1_faces[0]

# Get the face IDs in a second image
image_2_path = os.path.join('data', 'face', 'store_cam2.jpg')
image_2_stream = open(image_2_path, "rb")
image_2_faces = face_client.face.detect_with_stream(image=image_2_stream)
image_2_face_ids = list(map(lambda face: face.face_id, image_2_faces))

# Find faces in image 2 that are similar to the one in image 1
similar_faces = face_client.face.find_similar(face_id=face_1.face_id, face_ids=image_2_face_ids)

# Show the face in image 1, and similar faces in image 2(code in python_code/face.py)
faces.show_similar_faces(image_1_path, face_1, image_2_path, image_2_faces, similar_faces)

## 识别人脸

目前你已了解人脸服务可以检测人脸和面部特征，并且可以识别出两张相似的脸。你可再进一步，实现一个面部识别解决方案，在其中训练人脸服务来识别特定人员的脸。此类解决方案的应用范围非常广泛，比如在社交媒体应用程序中自动标记朋友的照片，或者将面部识别作为生物特征身份验证系统的一部分。

要了解其中的原理，让我们假设 Northwind Traders 公司希望使用面部识别来确保只有 IT 部门中得到授权的员工才能访问安全系统。

首先，我们要创建一个人员组来代表这些得到授权的员工。

In [ ]:
group_id = 'employee_group_id'
try:
    # Delete group if it already exists
    face_client.person_group.delete(group_id)
except Exception as ex:
    print(ex.message)
finally:
    face_client.person_group.create(group_id, 'employees')
    print ('Group created!')

创建好该*人员*组后，就能为其中应该包括的每位员工添加一个对应的“*人员*”，然后为每个人员录入多张照片，从而让人脸服务学习每个人独特的面部特征。提供的图像最好是能展示同一个人不同的姿势和不同的面部表情。

我们会添加一个名叫 Wendell 的员工，并录入该员工的三张照片。

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

# Add a person (Wendell) to the group
wendell = face_client.person_group_person.create(group_id, 'Wendell')

# Get photo's of Wendell
folder = os.path.join('data', 'face', 'wendell')
wendell_pics = os.listdir(folder)

# Register the photos
i = 0
fig = plt.figure(figsize=(8, 8))
for pic in wendell_pics:
    # Add each photo to person in person group
    img_path = os.path.join(folder, pic)
    img_stream = open(img_path, "rb")
    face_client.person_group_person.add_face_from_stream(group_id, wendell.person_id, img_stream)

    # Display each image
    img = Image.open(img_path)
    i +=1
    a=fig.add_subplot(1,len(wendell_pics), i)
    a.axis('off')
    imgplot = plt.imshow(img)
plt.show()

添加人员并录入照片后，即可训练人脸服务来识别各个人员。

In [ ]:
face_client.person_group.train(group_id)
print('Trained!')

现在，你可以使用训练好的模型在图像中识别已录入的人脸。

In [ ]:
# Get the face IDs in a second image
image_path = os.path.join('data', 'face', 'employees.jpg')
image_stream = open(image_path, "rb")
image_faces = face_client.face.detect_with_stream(image=image_stream)
image_face_ids = list(map(lambda face: face.face_id, image_faces))

# Get recognized face names
face_names = {}
recognized_faces = face_client.face.identify(image_face_ids, group_id)
for face in recognized_faces:
    person_name = face_client.person_group_person.get(group_id, face.candidates[0].person_id).name
    face_names[face.face_id] = person_name

# show recognized faces
faces.show_recognized_faces(image_path, image_faces, face_names)



## 了解详细信息

要详细了解人脸认知服务，请参阅[人脸文档](https://docs.microsoft.com/azure/cognitive-services/face/)
